In [ ]:
import pandas as pd
import numpy as np
import os, sys
# import matplotlib.pyplot as plt
# import seaborn as sns
import json
sys.path.append('../')
from models.utils import rocksdb_knobs_make_dict

In [ ]:
PATH_internal = "../data/internal"
PATH_external = "../data/external"
PATH_knobs = "../data/rocksdb_conf"
wk_len = 16

In [ ]:
knobs = rocksdb_knobs_make_dict(PATH_knobs)
knobs = pd.DataFrame(data=knobs['data'].astype(np.float32), columns=knobs['columnlabels'])
columns = knobs.columns
knobs.head(4)

In [ ]:
wk_len = 16
internal_dict = {}

pruned_im = pd.read_csv(os.path.join(PATH_internal, 'internal_ensemble_pruned_tmp.csv'), index_col=0)
for wk in range(wk_len):
    im = pd.read_csv(os.path.join(PATH_internal, f'internal_results_{wk}.csv'), index_col=0)
    internal_dict[wk] = im[pruned_im.columns]
#     break
internal_dict[0].head(4)

In [ ]:
external_dict = {}
for wk in range(wk_len):
    ex = pd.read_csv(os.path.join(PATH_external, f'external_results_{wk}.csv'), index_col=0)
    external_dict[wk] = ex
    break
external_dict[0].head(4)

In [ ]:
from scipy.spatial import distance
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [ ]:
def euclidean_distance(a, b):
    res = a - b
    res = res ** 2
    res = np.sqrt(res)
#     return res
    return np.average(res)

In [ ]:
scaler = MinMaxScaler().fit(pd.concat(internal_dict))

In [ ]:
def get_euclidean_distance(wk_num, size):
    wk = []
    for im_d in internal_dict:
        wk.append(scaler.transform(internal_dict[im_d].iloc[:size, :]))
        
    big = 100
    for i in range(16):
        ed = euclidean_distance(wk[wk_num], wk[i])
        if ed<big and wk_num != i: 
            big=ed
            idx = i
        print(f'{i:4}th   {ed}')
    print('best similar workload is ',idx, 'th')

In [ ]:
get_euclidean_distance(0, 10)

In [ ]:
get_euclidean_distance(0, 50)

In [ ]:
get_euclidean_distance(0, 100)

In [ ]:
get_euclidean_distance(0, 1000)